In [1]:
# Set seeds for reproducible results
from numpy.random import seed
seed(327)
import tensorflow
tensorflow.random.set_seed(327)

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

import itertools

Using TensorFlow backend.


#### Helper Functions

##### 1. Data Preprocessing

In [2]:
def preprocess_data(raw_data):
    
    # Add categorical dummy variables (All 0s represent)
    tasknum_dummies = pd.get_dummies(raw_data['Task_num'],
                                     prefix="TaskNum") # Create dummy variables
    data = pd.concat([raw_data, tasknum_dummies], axis=1) 

    # Remove the unnecessary columns
    remove_cols = ["Skip_distance",
              "Subject",
              "Mean_fixation_duration",
              "Loag_Fixationtime",
              "Log_timetoF",
              "Task_completion_duration",
              "Compressed_scanpath_value", 
              "Total_r_d",
              "Compressed_M_Minimal",
              "Strictly_linearWID",
              "Mean_fixation_duration_for_onelink",
              "Skip",
              "Skip_count", 
              "Task_num",
              "TaskNum_t9"]  # Remove one dummy variable to avoid the dummy variable trap

    data = data.drop(remove_cols, axis=1)
    
    # Encode the Screen_size column
    vals = ['S', 'M', 'L']
    for i in range(len(vals)):
        data.at[data['Screen_size'] == vals[i], ['Screen_size']] = i    

    # Replace missing values with 0 in column Regression_distance
    preprocessed_data = data.fillna(0)

    # Inspect the number of missing values in the preprocessed_data dataset
    num_missing = preprocessed_data.isnull().sum().sum()
    print("The number of missing values in the data = {}".format(num_missing))
    print("Number of features remaining = {}".format(data.shape[1]))
    
    return preprocessed_data

#### Data Preparation

In [3]:
# Step 1. Import the dataset

# Total number of columns in the dataset = 36
required_cols = list(range(36))

# Read the dataset
raw_data = pd.read_excel("Jae-Second_Exp_data.xlsx",
                     sheet_name="Analysis_summary",
                     nrows=161,
                     usecols = required_cols)

# Step 2. Preprocess the data
data = preprocess_data(raw_data = raw_data)

# Step 3. Split the data into training and test sets

# Divide into features and target variables
X = data.drop("Screen_size", axis=1)
y = data['Screen_size']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Normalise training data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Perform PCA
pca = PCA()
X_train_scaled_pca = pca.fit_transform(X_train_scaled)
X_test_scaled_pca = pca.transform(X_test_scaled)

x_train = X_train_scaled_pca[:,[9,10,26,27]]
x_test = X_test_scaled_pca[:,[9,10,26,27]]

The number of missing values in the data = 0
Number of features remaining = 30


#### Import best model parameters

In [4]:
data = pd.read_csv("MY BEST NETWORKS")

best_models = data[data.fitness_scores > 0.467]
best_models.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   num_hidden_layers  24 non-null     float64
 1   neurons_per_layer  24 non-null     float64
 2   dropout_rate       24 non-null     float64
 3   fitness_scores     24 non-null     float64
dtypes: float64(4)
memory usage: 960.0 bytes


#### 1. Activation Function = Softsign

In [5]:
# Define the network (using the parameters from best models)

# Perform cross-validation on the network and store the results

In [6]:
def build_nn(num_hidden_layers, neurons_per_layer):
#     activation_function = 'softsign'
#     optimizer = 'adagrad'

    # Initialising the ANN
    classifier = Sequential()

    # Add first hidden layer
    classifier.add(Dense(units = neurons_per_layer, activation = activation_function, input_dim = x_train.shape[1]))

    # Add hidden layers
    for i in range(num_hidden_layers - 1):
        classifier.add(Dense(units = neurons_per_layer, activation = activation_function))
        classifier.add(Dropout(dropout, seed=327))

    # Adding the output layer
    classifier.add(Dense(units = 3, activation = 'softmax'))

    # Compiling the ANN
    classifier.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier

In [7]:
def get_individual_fitness():
    
    model_params = {'num_hidden_layers':[num_hidden_layers], 
                    'neurons_per_layer':[neurons_per_layer]}    
    # create model
    model = KerasClassifier(build_fn=build_nn, epochs=20, batch_size=1, verbose=0)    
    # Perform k-fold cross validation (using GridSearch here to reduce code size)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=327)
    model_cv = GridSearchCV(estimator=model, param_grid=model_params, scoring='f1_weighted', cv=kfold, n_jobs=-1)
    model_cv.fit(x_train, y_train)
    
    # Return weighted F1-score 
    return model_cv.cv_results_['mean_test_score'][0]

In [17]:
results = []
for i in range(len(best_models)):

    # Get parameters from individual
    num_hidden_layers, neurons_per_layer, dropout = best_models.iloc[i,:-1].values
    num_hidden_layers, neurons_per_layer = int(num_hidden_layers), int(neurons_per_layer)
    activation_function = 'tanh'
    optimizer = 'adagrad'
    
    results.append(get_individual_fitness())
    print(i+1)
    
results_df = pd.DataFrame({'optimizer':[optimizer] * len(best_models),
          'activation_function':[activation_function] * len(best_models),
          'fitness_scores':results})

a = pd.concat([best_models.drop('fitness_scores',axis=1), results_df], axis=1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [22]:
def approach_new(af, opt):
    
    results = []
    for i in range(len(best_models)):

        # Get parameters from individual
        num_hidden_layers, neurons_per_layer, dropout = best_models.iloc[i,:-1].values
        num_hidden_layers, neurons_per_layer = int(num_hidden_layers), int(neurons_per_layer)
        activation_function = af
        optimizer = opt

        results.append(get_individual_fitness())
        print(i+1)

    results_df = pd.DataFrame({'optimizer':[optimizer] * len(best_models),
              'activation_function':[activation_function] * len(best_models),
              'fitness_scores':results})

    a = pd.concat([best_models.drop('fitness_scores',axis=1), results_df], axis=1)
    
    return a

In [31]:
final = pd.DataFrame()

for af in ['relu', 'tanh', 'selu', 'softsign']:
    for opt in ['sgd', 'rmsprop', 'adam', 'adadelta', 'adagrad', 'adamax', 'nadam']:
        
        a = approach_new(af, opt)
        
        final = final.append(a)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
2

In [32]:
# final.to_csv("best_models_and_activation_optimizers.csv")

In [33]:
final

,num_hidden_layers,neurons_per_layer,dropout_rate,optimizer,activation_function,fitness_scores
0,2.0,93.0,0.357395,sgd,relu,0.376386
1,2.0,93.0,0.199754,sgd,relu,0.372747
2,2.0,70.0,0.159685,sgd,relu,0.381415
3,6.0,85.0,0.172981,sgd,relu,0.395266
4,2.0,75.0,0.186952,sgd,relu,0.357958
...,...,...,...,...,...,...
19,3.0,87.0,0.199754,nadam,softsign,0.399536
20,2.0,89.0,0.199754,nadam,softsign,0.375168
21,2.0,85.0,0.014486,nadam,softsign,0.362382
22,2.0,89.0,0.208475,nadam,softsign,0.402751


In [45]:
final.sort_values(by='fitness_scores', ascending=False)

,num_hidden_layers,neurons_per_layer,dropout_rate,optimizer,activation_function,fitness_scores
3,6.0,85.0,0.172981,adagrad,softsign,0.442106
4,2.0,75.0,0.186952,adamax,selu,0.433926
8,3.0,85.0,0.123161,sgd,tanh,0.427011
3,6.0,85.0,0.172981,adagrad,relu,0.426135
16,3.0,97.0,0.339876,rmsprop,softsign,0.419735
...,...,...,...,...,...,...
8,3.0,85.0,0.123161,adam,softsign,0.338395
9,5.0,75.0,0.172981,adadelta,relu,0.337989
12,2.0,85.0,0.014486,sgd,selu,0.337715
22,2.0,89.0,0.208475,rmsprop,softsign,0.336033


In [47]:
def build_nn(num_hidden_layers=6, neurons_per_layer=85):
    
    activation_function = 'softsign'
    optimizer = 'adagrad'

    # Initialising the ANN
    classifier = Sequential()

    # Add first hidden layer
    classifier.add(Dense(units = neurons_per_layer, activation = activation_function, input_dim = x_train.shape[1]))

    # Add hidden layers
    for i in range(num_hidden_layers - 1):
        classifier.add(Dense(units = neurons_per_layer, activation = activation_function))
        classifier.add(Dropout(dropout, seed=327))

    # Adding the output layer
    classifier.add(Dense(units = 3, activation = 'softmax'))

    # Compiling the ANN
    classifier.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
   
    return classifier

In [55]:
def get_individual_fitness():
    
    model_params = {'num_hidden_layers':[6], 
                    'neurons_per_layer':[85]}    
    # create model
    model = KerasClassifier(build_fn=build_nn, epochs=100, batch_size=5, verbose=1)    
    # Perform k-fold cross validation (using GridSearch here to reduce code size)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=327)
    model_cv = GridSearchCV(estimator=model, param_grid=model_params, scoring='f1_weighted', cv=kfold)
    model_cv.fit(x_train, y_train)
    
    # Return weighted F1-score 
    return model_cv.cv_results_['mean_test_score'][0]

In [56]:
get_individual_fitness()

Epoch 1/100
115/115 [==============================] - 0s 3ms/step - loss: 1.1607 - accuracy: 0.3130
Epoch 2/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0894 - accuracy: 0.3913
Epoch 3/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0776 - accuracy: 0.3478
Epoch 4/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0648 - accuracy: 0.4348
Epoch 5/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0684 - accuracy: 0.4348
Epoch 6/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0710 - accuracy: 0.4087
Epoch 7/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0724 - accuracy: 0.4261
Epoch 8/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0428 - accuracy: 0.4696
Epoch 9/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0610 - accuracy: 0.4174
Epoch 10/100
115/115 [==============================] - 0s 1ms/step - loss: 1.0645 - accura

0.3594087394087394